## REACTIV example

### Dataset for the example

Get the dataset from https://github.com/elisekoeniguer/REACTIV/tree/master/REACTIV/Datasets

In [1]:
import glob 
from skimage import io
import numpy as np
import gdal 
from reactiv import *
import datetime
import os
%load_ext autoreload
%autoreload 2

In [6]:
l = glob.glob('/workspace/REACTIV/REACTIV/Datasets//*VH_amplitude.tif') 

In [7]:
l

['/workspace/REACTIV/REACTIV/Datasets/20150323_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20160105_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20151224_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20160210_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20160305_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20150721_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20150428_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20160329_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20150416_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20160129_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20160317_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20151212_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20150404_IW1_VH_amplitude.tif',
 '/workspace/REACTIV/REACTIV/Datasets/20150627_IW1_VH_amplitude.tif',
 '/workspace/REACTIV

Create a numpy stack with the dataset

In [8]:
im=io.imread(l[0])
nx=im.shape[0]
ny=im.shape[1]

my_stack = []

for i in l:
    my_stack.append(io.imread(i))

In [4]:
np.stack(my_stack)[2,:,:]

array([[  3.67375493e-01,   2.77353197e-01,   2.12059900e-01, ...,
          2.20561619e+01,   9.90622234e+00,   8.00498295e+00],
       [  2.17653465e+00,   1.27446938e+00,   8.16548586e-01, ...,
          3.15866280e+01,   4.32818108e+01,   3.40026970e+01],
       [  5.67372847e+00,   4.34832811e+00,   3.23996425e+00, ...,
          6.69767532e+01,   1.51723953e+02,   1.58831268e+02],
       ..., 
       [  1.12617695e+00,   1.25495207e+00,   1.05415249e+00, ...,
          1.08297195e+02,   6.99669952e+01,   6.24757118e+01],
       [  1.32070923e+00,   9.38759565e-01,   7.44275630e-01, ...,
          1.11019325e+02,   6.34804955e+01,   3.25577240e+01],
       [  3.33532363e-01,   1.26941621e-01,   1.69315591e-01, ...,
          5.44859123e+01,   4.16544685e+01,   1.46829481e+01]], dtype=float32)

Create the date vector

In [11]:
# Compute the date vector
datenum=[]
for i in range(0, len(l)):
   
    (filepath, filename) = os.path.split(l[i])
    year=(int(filename[0:4]))
    month=(int(filename[4:6]))
    day=(int(filename[6:8]))
    d1 = datetime.date(year,month,day)
    datenum.append(d1.toordinal())


In [12]:
datenum

[735680,
 735968,
 735956,
 736004,
 736028,
 735800,
 735716,
 736052,
 735704,
 735992,
 736040,
 735944,
 735692,
 735776,
 735980,
 735656]

REACTIV

In [13]:
res = reactiv(np.stack(my_stack), datenum)

/workspace/REACTIV/REACTIV/Python/reactiv/reactiv/__init__.py:49: RuntimeWarning: divide by zero encountered in divide
  R = (R - CV) / (alpha / np.sqrt(N)) / 10.0 + 0.25;
/workspace/REACTIV/REACTIV/Python/reactiv/reactiv/__init__.py:50: RuntimeWarning: invalid value encountered in multiply
  R = (R > 1) * np.ones([nx, ny]) + (R < 1) * R;   # Cast Coefficient of Varation R max to 1.
/workspace/REACTIV/REACTIV/Python/reactiv/reactiv/__init__.py:51: RuntimeWarning: invalid value encountered in greater
  R = (R > 0) * R;
/workspace/REACTIV/REACTIV/Python/reactiv/reactiv/__init__.py:51: RuntimeWarning: invalid value encountered in multiply
  R = (R > 0) * R;


In [14]:
res

array([[[ 0.3930367 ,         nan,         nan],
        [ 0.14967169,         nan,         nan],
        [        nan,  0.18797113,         nan],
        ..., 
        [ 0.25520798,         nan,         nan],
        [ 0.2034805 ,         nan,         nan],
        [ 0.21406481,         nan,         nan]],

       [[        nan,  0.29788162,         nan],
        [        nan,  0.19508421,         nan],
        [        nan,  0.37421399,         nan],
        ..., 
        [        nan,         nan,  0.37499159],
        [        nan,         nan,  0.54858432],
        [ 0.37040054,         nan,         nan]],

       [[ 0.38219665,         nan,         nan],
        [        nan,  0.20875318,         nan],
        [ 0.36218155,         nan,         nan],
        ..., 
        [        nan,         nan,  0.60583923],
        [ 0.90617276,         nan,         nan],
        [        nan,         nan,  0.61931733]],

       ..., 
       [[ 0.31493456,         nan,         nan],
        

Save result

In [16]:
outfile = 'reactiv.tif'

driver = gdal.GetDriverByName('GTiff')

outdata = driver.Create(outfile, 
                        res.shape[1], 
                        res.shape[0], 
                        3,
                        gdal.GDT_Byte) 



for band_index in range(0, res.shape[2]):
    
    outband = outdata.GetRasterBand(band_index+1)
   
    outband.WriteArray(res[:,:,band_index]*255)

outdata.FlushCache()